In [13]:
import os
import sys
from pathlib import Path

import numpy as np

# Ensure the parent directory is in the system path for module imports
sys.path.append(os.path.abspath(os.path.join("..")))

from utils import load_car_data

In [ ]:
DATA_PATH = Path("./../data/fuel_consumption.csv")
df = load_car_data(DATA_PATH)

numerical_features = ["release_year", "gears", "engine_size", "cylinders"]
categorical_features = ["make", "vehicle_class", "fuel_type", "transmission_type"]
features = numerical_features + categorical_features

targets = ["emissions", "fc_mixed", "fc_city", "fc_highway"]

X = df[features]
Y = df[targets]

### Model definition

In [15]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

preprocessor = ColumnTransformer(
    transformers=[
        ("categorical", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        (
            "numerical",
            Pipeline(
                steps=[
                    ("imputer", SimpleImputer(strategy="mean")),
                    ("scaler", StandardScaler()),
                ]
            ),
            numerical_features,
        ),
    ]
)

pipeline = Pipeline(
    steps=[("preprocessor", preprocessor), ("regressor", Lasso(max_iter=10000))]
)

param = {"regressor__alpha": np.logspace(-4, 4, 5)}

In [16]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipeline, param, cv=5, n_jobs=-1, refit=True)
grid.fit(X, Y)

print(f"Best parameters : {grid.best_params_}")
print(f"Best score : {grid.best_score_:0.2f}")

/Users/mathisderenne/.local/share/uv/python/cpython-3.12.8-macos-aarch64-none/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/Users/mathisderenne/.local/share/uv/python/cpython-3.12.8-macos-aarch64-none/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/Users/mathisderenne/.local/share/uv/python/cpython-3.12.8-macos-aarch64-none/lib/python3.12/multiprocessing/queues.py:122: UserWarning: pkg_resources is deprecated as an API. See https://setuptoo

Best parameters : {'regressor__alpha': np.float64(0.0001)}
Best score : 0.77


### Save model

In [18]:
import joblib

joblib.dump(grid.best_estimator_, "../data/lasso_regression.pkl")

['../data/lasso_regression.pkl']